# Bob Ross Continued
## Attempting to predict season based on transcript
#### Julian Furrow & Paul Kauffman

# [T1]

## Load Data & Predict

In [258]:
import pandas as pd
from __future__ import division
import numpy as np 
from nltk.corpus import stopwords
from plotly.offline import init_notebook_mode, iplot
import nltk
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
import plotly
plotly.offline.init_notebook_mode(connected=True)

### Most EDA was done with last homework, but a quick transformation and look at the data again never hurts.

In [259]:
df = pd.read_csv('C:/Users/Paul/Desktop/Rockhurst/BIA 6304-Text Mining/Homework 3/bob_ross_episodes_clean.csv')
df = df.replace('/n', ' ', regex = True) #already considered and addressed in homework 2
print(df.shape)
print(list(df)) #headers, 'transcript' contains the episode text
df.head(1) #looking at the first episode transcript to see how the text looks

(402, 5)
['Episode Name ', 'Season_#', 'Episode_#', 'snippet.title', 'transcript']


,Episode Name,Season_#,Episode_#,snippet.title,transcript
0,Island in the Wilderness,29,1,Bob Ross - Island in the Wilderness (Season 29...,"(mellow smooth jazz music) - Hello, I'm Bob Ro..."


#### Replacing unneeded characters and replacing with spaces

In [260]:
df.columns = df.columns.str.replace('_#','') 
df.head(1)

,Episode Name,Season,Episode,snippet.title,transcript
0,Island in the Wilderness,29,1,Bob Ross - Island in the Wilderness (Season 29...,"(mellow smooth jazz music) - Hello, I'm Bob Ro..."


#### Removing NA value transcripts

In [261]:
print(len(df))
df = df.dropna()
print(len(df))

402
249


#### Lowercasing all text in "transcript"

In [262]:
df['transcript']= df['transcript'].str.lower()
df.head()

,Episode Name,Season,Episode,snippet.title,transcript
0,Island in the Wilderness,29,1,Bob Ross - Island in the Wilderness (Season 29...,"(mellow smooth jazz music) - hello, i'm bob ro..."
1,Mystic Mountain,20,1,Bob Ross - Mystic Mountain (Season 20 Episode 1),"(instrumental music) - hello, i'm bob ross, an..."
2,Mountain Lake Falls,29,6,Bob Ross - Mountain Lake Falls (Season 29 Epis...,"- hi, welcome back. certainly glad you could j..."
3,Arctic Beauty,6,7,Bob Ross - Arctic Beauty (Season 6 Episode 7),- welcome back. awful glad you could join me t...
4,A Walk in the Woods,1,1,Bob Ross - A Walk in the Woods (Season 1 Episo...,"- hi, i'm bob ross, and for the next 13\nweeks..."


## Creating Word Count by episode
    looking to see if there is any difference in word count by season

In [263]:
df['word_count'] = df['transcript'].apply(lambda x : len(x.split())) #wordcount

In [264]:
df.head()

,Episode Name,Season,Episode,snippet.title,transcript,word_count
0,Island in the Wilderness,29,1,Bob Ross - Island in the Wilderness (Season 29...,"(mellow smooth jazz music) - hello, i'm bob ro...",4060
1,Mystic Mountain,20,1,Bob Ross - Mystic Mountain (Season 20 Episode 1),"(instrumental music) - hello, i'm bob ross, an...",4286
2,Mountain Lake Falls,29,6,Bob Ross - Mountain Lake Falls (Season 29 Epis...,"- hi, welcome back. certainly glad you could j...",3327
3,Arctic Beauty,6,7,Bob Ross - Arctic Beauty (Season 6 Episode 7),- welcome back. awful glad you could join me t...,3431
4,A Walk in the Woods,1,1,Bob Ross - A Walk in the Woods (Season 1 Episo...,"- hi, i'm bob ross, and for the next 13\nweeks...",3137


### Episodes by Season

In [265]:
episode_count = df.groupby(['Season'])[['Episode']].count()
x = episode_count.index
y = episode_count.Episode

data = [go.Bar(x=x,
            y=y,
              text = y,
               textposition = 'outside',
              opacity =.9)]
layout = go.Layout(
                   title='Count of Episodes by Season',
                   xaxis=dict(title='Season'),
                   yaxis=dict( title='Episode Count'))
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='bar-direct-labels')

### Average wordcount by season

In [266]:
season_word_sum=  df.groupby(['Season'])[['word_count']].sum()
season_avg = pd.DataFrame(round(season_word_sum['word_count']/episode_count['Episode']))
season_avg.columns = ['Episode']

x = season_avg.index
y = season_avg.Episode

data = [go.Bar(x=x,
            y=y,
              text = y,
               textposition = 'auto',
               textfont = dict(color = 'rgb(225,225,225)'),
                  marker=dict(
        color='rgb(158,5,225)',
        line=dict(
            color='rgb(10,48,107)',
            width=1.5,
        )
    ),              
              opacity =.9)]
layout = go.Layout(
                   title='Avg. Word Count by Season',
                   xaxis=dict(title='Season'),
                   yaxis=dict( title='Avg. Word Count'))
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='bar-direct-labels')



#### Because of the lack of episodes in season 12 & 17 the decision was made to remove these seasons because of a lack of observations

In [267]:
print(df.shape)
#dropping season 12 & 17
df2 = df[(df.Season != 12) & (df.Season != 17)]
#df2 = df2.drop(df2[df2.Season == 17].index, axis=0, inplace=True)
print(df2.shape)

(249, 6)
(245, 6)


In [268]:
season_word_sum['word_count'].sum()

953840

### Final dataframe

In [269]:
print(df2.shape)
df2.head(10)

(245, 6)


,Episode Name,Season,Episode,snippet.title,transcript,word_count
0,Island in the Wilderness,29,1,Bob Ross - Island in the Wilderness (Season 29...,"(mellow smooth jazz music) - hello, i'm bob ro...",4060
1,Mystic Mountain,20,1,Bob Ross - Mystic Mountain (Season 20 Episode 1),"(instrumental music) - hello, i'm bob ross, an...",4286
2,Mountain Lake Falls,29,6,Bob Ross - Mountain Lake Falls (Season 29 Epis...,"- hi, welcome back. certainly glad you could j...",3327
3,Arctic Beauty,6,7,Bob Ross - Arctic Beauty (Season 6 Episode 7),- welcome back. awful glad you could join me t...,3431
4,A Walk in the Woods,1,1,Bob Ross - A Walk in the Woods (Season 1 Episo...,"- hi, i'm bob ross, and for the next 13\nweeks...",3137
6,Campfire,3,10,Bob Ross - Campfire (Season 3 Episode 10),- welcome back. glad to see ya today. you know...,3250
7,Valley View,21,1,Bob Ross - Valley View (Season 21 Episode 1),"(bright music) - hello, i'm bob ross and\ni'd ...",4578
8,Lazy River,2,10,Bob Ross - Lazy River (Season 2 Episode 10),"- welcome back, glad to see you again. thought...",2975
9,Fisherman's Trail,28,1,Bob Ross - Fisherman's Trail (Season 28 Episod...,(upbeat music) - hello i'm bob ross and i'd li...,3993
11,Shades of Grey,2,4,Bob Ross - Shades of Grey (Season 2 Episode 4),"- all right, here we\nare ready to go again. y...",3234


### Creating count and weighted vectors

In [270]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv1 = CountVectorizer(lowercase=True, 
                     stop_words='english',
                     binary=False,
                     max_df=0.95, 
                     min_df=0.05,
                     ngram_range = (1,2)) 

tfidf1 = TfidfVectorizer(lowercase=True, 
                        stop_words= 'english', 
                        max_df=0.95, 
                        min_df=0.05,
                        use_idf = True,
                        ngram_range = (1,2))

cv_dm = cv1.fit_transform(df2['transcript'])

tfidf_dm = tfidf1.fit_transform(df2['transcript'])

print(cv_dm.shape)
print(tfidf_dm.shape)

(245, 5334)
(245, 5334)


#### Test Train Split

In [271]:
# train test split
# data are X, labels are y
from sklearn.model_selection import train_test_split, cross_val_score

X = cv_dm.toarray()  #remember this is the output from the vectorizer and we are turning it into an array
#print(type(X), X[0:10])


y = df2['Season'].values #this is an array of labels
#print(type(y), y[0:10])


X2 = tfidf_dm.toarray()  #remember this is the output from the vectorizer and we are turning it into an array
#print(type(X2), X2[0:10])


y2 = df2['Season'].values #this is an array of labels
#print(type(y2), y2[0:10])

In [272]:
# train and test on count
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42) #random_state is set seed

# function creates 4 output structures - order matters
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 5334)
(13, 5334)
(232,)
(13,)


In [273]:
# train and test on weights
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.05, random_state=42) #random_state is set seed

# function creates 4 output structures - order matters
print(X2_train.shape)
print(X2_test.shape)
print(y2_train.shape)
print(y2_test.shape)

(232, 5334)
(13, 5334)
(232,)
(13,)


## Decision Tree
    & Cross Valadation

In [274]:
#Decision Tree with count

from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

# fit a Decision Tree dt_model_1 to the data
dt_model_1 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
#print(dt_model_1)
dt_model_1.fit(X_train, y_train)

# make predictions
clf1_expected = y_test
clf1_predicted = dt_model_1.predict(X_test)

#print(dt_model_1.score(X_test, y_test))

# summarize the fit of the dt_model_1
print("accuracy: \n" + str(metrics.accuracy_score(clf1_expected, clf1_predicted)))
print('===================')
print(metrics.classification_report(clf1_expected, clf1_predicted))

accuracy: 
0.23076923076923078
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
         18       0.00      0.00      0.00         2
         19       0.00      0.00      0.00         1
         20       1.00      1.00      1.00         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.50      1.00      0.67         1
         24       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         1
         29       0.00      0.00      0.00         0
         30       0.00      0.00      0.00         0
         31       0.00      0.00      0.00         0

avg / total       0.15      0.23      0.18        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



#### Cross validation 

In [275]:
#verify DT with Cross Validation
scores = cross_val_score(dt_model_1, X_train, y_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.25641026 0.18421053 0.27777778 0.17647059 0.24       0.35
 0.2        0.2       ]


0.23560864359238973

In [276]:
#Decision Tree with weights

from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

# fit a Decision Tree dt_model_2 to the data
dt_model_2 = DecisionTreeClassifier(random_state = 42)
#print(dt_model_2)
dt_model_2.fit(X2_train, y2_train)

# make predictions
clf2_eX2pected = y2_test
clf2_predicted = dt_model_2.predict(X2_test)

#print(dt_model_2.score(X2_test, y2_test))

# summarize the fit of the dt_model_2
print("accuracy: \n" + str(metrics.accuracy_score(clf2_eX2pected, clf2_predicted)))
print('===================')
print(metrics.classification_report(clf2_eX2pected, clf2_predicted))

accuracy: 
0.38461538461538464
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         1
          2       0.00      0.00      0.00         1
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
         18       1.00      1.00      1.00         2
         19       1.00      1.00      1.00         1
         20       1.00      1.00      1.00         1
         21       0.00      0.00      0.00         1
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       1.00      1.00      1.00         1
         29       0.00      0.00      0.00         0
         30       0.00      0.00      0.00         0

avg / total       0.38      0.38      0.38        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [277]:
#verify DT with Cross Validation
scores = cross_val_score(dt_model_2, X2_train, y2_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.30769231 0.10526316 0.13888889 0.20588235 0.2        0.3
 0.3        0.3       ]


0.23221583842713875

## Logistic Regression 
    & Cross Valadation

In [278]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

# fit a logistic regression lr_model to the data
lr_model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
#print(lr_model)
lr_model.fit(X_train, y_train)

# make predictions
clf3_expected = y_test
clf3_predicted = lr_model.predict(X_test)

#print(lr_model.score(X_test, y_test))

# summarize the fit of the lr_model
print("accuracy: \n" + str(metrics.accuracy_score(clf3_expected, clf3_predicted)))
print('===================')
print(metrics.classification_report(clf3_expected, clf3_predicted))

accuracy: 
0.46153846153846156
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.50      1.00      0.67         1
         18       1.00      0.50      0.67         2
         19       0.00      0.00      0.00         1
         20       1.00      1.00      1.00         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.00      0.00      0.00         1
         24       1.00      1.00      1.00         1
         27       0.50      1.00      0.67         1
         28       0.00      0.00      0.00         1

avg / total       0.42      0.46      0.41        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [279]:
#verify DT with Cross Validation
scores = cross_val_score(lr_model, X_train, y_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.46153846 0.47368421 0.36111111 0.61764706 0.68       0.65
 0.5        0.55      ]


0.5367476052499273

In [280]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

# fit a logistic regression lr_model2 to the data
lr_model2 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
#print(lr_model2)
lr_model2.fit(X2_train, y2_train)

# make predictions
clf3_expected = y2_test
clf3_predicted = lr_model2.predict(X2_test)

#print(lr_model2.score(X2_test, y2_test))

# summarize the fit of the lr_model2
print("accuracy: \n" + str(metrics.accuracy_score(clf3_expected, clf3_predicted)))
print('===================')
print(metrics.classification_report(clf3_expected, clf3_predicted))

accuracy: 
0.23076923076923078
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.50      1.00      0.67         1
         18       0.00      0.00      0.00         2
         19       0.00      0.00      0.00         1
         20       0.50      1.00      0.67         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         1
         29       0.00      0.00      0.00         0
         30       0.00      0.00      0.00         0

avg / total       0.12      0.23      0.15        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [281]:
#verify DT with Cross Validation
scores = cross_val_score(lr_model2, X2_train, y2_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.30769231 0.36842105 0.22222222 0.32352941 0.44       0.45
 0.3        0.35      ]


0.3452331242888519

## Naive Bayes 
    & Cross Valadation

In [282]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# fit a Naive Bayes model to the data
nb_model = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
#print(nb_model)
nb_model.fit(X_train, y_train)

# make predictions
clf5_expected = y_test
clf5_predicted = nb_model.predict(X_test)

#print(nb_model.score(X_test, y_test))

# summarize the fit of the nb_model
print("accuracy: \n" + str(metrics.accuracy_score(clf5_expected, clf5_predicted)))
print('===================')
print(metrics.classification_report(clf5_expected, clf5_predicted))

accuracy: 
0.38461538461538464
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.50      1.00      0.67         1
         18       1.00      0.50      0.67         2
         19       0.00      0.00      0.00         1
         20       1.00      1.00      1.00         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         27       1.00      1.00      1.00         1
         28       0.00      0.00      0.00         1
         30       0.00      0.00      0.00         0

avg / total       0.38      0.38      0.36        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [283]:
#verify DT with Cross Validation
scores = cross_val_score(nb_model, X_train, y_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.43589744 0.47368421 0.30555556 0.35294118 0.56       0.75
 0.3        0.5       ]


0.45975979730623695

In [284]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# fit a Naive Bayes model to the data
nb2_model = MultinomialNB()
print(nb2_model)
nb2_model.fit(X2_train, y2_train)

# make predictions
clf6_expected = y2_test
clf6_predicted = nb2_model.predict(X2_test)

#print(nb2_model.score(X2_test, y2_test))

# summarize the fit of the nb2_model
print("accuracy: \n" + str(metrics.accuracy_score(clf6_expected, clf6_predicted)))
print('===================')
print(metrics.classification_report(clf6_expected, clf6_predicted))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
accuracy: 
0.15384615384615385
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       1.00      1.00      1.00         1
         18       0.00      0.00      0.00         2
         19       0.00      0.00      0.00         1
         20       0.00      0.00      0.00         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         1
         30       0.00      0.00      0.00         0

avg / total       0.12      0.15      0.13        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [285]:
#verify NB2 with Cross Validation
scores = cross_val_score(nb2_model, X2_train, y2_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.28205128 0.28947368 0.22222222 0.32352941 0.4        0.3
 0.15       0.25      ]


0.27715957503109206

#verify DT with Cross Validation
scores = cross_val_score(rf3_model, X3_train, y3_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

# [T2 ]

 ## trying to improve the predictions through PCA, New Vectors, and Custom Stop Words.

### New Weights Vectorization
Minimize the feature space

In [286]:
tfidf2 = TfidfVectorizer(lowercase=True, 
                        stop_words= 'english', 
                        max_df=0.9, 
                        min_df=0.15,
                        use_idf = True,
                        ngram_range = (1,2))

tfidf_dm2 = tfidf2.fit_transform(df2['transcript'])

print(tfidf_dm2.shape)


(245, 1806)


In [287]:
X3 = tfidf_dm2.toarray()  #remember this is the output from the vectorizer and we are turning it into an array
#print(type(X), X[0:10])


y3 = df2['Season'].values #this is an array of labels
#print(type(y), y[0:10])

### PCA with Weight Vectorizor

In [288]:
from sklearn.decomposition import PCA, RandomizedPCA

# we want something that is smaller than 1800
# let's start with 200 - that's about how big the second round space was
# maybe we've kept more info for the same number of features
#names = tfidf2.get_feature_names()

for i in range(0,300, 10):

    pca = PCA(n_components=i)
    X_pca = pca.fit_transform(tfidf_dm2.toarray())
    #print("Number of components: " + str(i))
    #print('PCA Total Variance Explained: ' + str(sum(pca.explained_variance_ratio_)))

    # Decided to use 140

In [289]:
# 140 components from above was determined to be the most valuable combination
pca = PCA(n_components=140)
X_pca = pca.fit_transform(tfidf_dm2.toarray())

print( 'PCA Total Variance Explained: ' + str(sum(pca.explained_variance_ratio_)))

PCA Total Variance Explained: 0.8431531272943045


In [290]:
X4 = X_pca  #remember this is the output from the vectorizer and we are turning it into an array
#print(type(X4), X4[0:10])


y4 = df2['Season'].values #this is an array of labels
#print(type(y4), y4[0:10])

In [291]:
# train and test on count
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.05, random_state=42) #random_state is set seed

# function creates 4 output structures - order matters
print(X4_train.shape)
print(X4_test.shape)
print(y4_train.shape)
print(y4_test.shape)

(232, 140)
(13, 140)
(232,)
(13,)


#### Random Forest using PCA variables derived from New Weighted Vector

In [292]:
# Random Forest using weights
from sklearn.ensemble import RandomForestClassifier
rf4_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

#print(rf4_model)

rf4_model.fit(X4_train, y4_train)

rf4_expected = y4_test
rf4_predicted = rf4_model.predict(X4_test)

#print(rf4_model.score(X4_test,y4_test))

# summarize the fit of the rf4_model
print("accuracy: \n" + str(metrics.accuracy_score(rf4_expected, rf4_predicted)))
print(metrics.classification_report(rf4_expected, rf4_predicted))

accuracy: 
0.23076923076923078
             precision    recall  f1-score   support

          1       0.25      1.00      0.40         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
         18       1.00      0.50      0.67         2
         19       0.00      0.00      0.00         1
         20       0.50      1.00      0.67         1
         21       0.00      0.00      0.00         1
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         25       0.00      0.00      0.00         0
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         1
         30       0.00      0.00      0.00         0

avg / total       0.21      0.23      0.18        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [293]:
#verify DT with Cross Validation
scores = cross_val_score(rf4_model, X4_train, y4_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.28205128 0.21052632 0.19444444 0.32352941 0.36       0.4
 0.2        0.3       ]


0.2838189317562383

#### Logistic Regression using PCA values derived from New Weighted Vector

In [294]:
from sklearn.linear_model import LogisticRegression

# fit a logistic regression lr_model2 to the data
lr_model3 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
#print(lr_model2)
lr_model3.fit(X4_train, y4_train)

# make predictions
clf7_expected = y4_test
clf7_predicted = lr_model3.predict(X4_test)

#print(lr_model2.score(X2_test, y2_test))

# summarize the fit of the lr_model2
print("accuracy: \n" + str(metrics.accuracy_score(clf7_expected, clf7_predicted)))
print('===================')
print(metrics.classification_report(clf7_expected, clf7_predicted))

accuracy: 
0.23076923076923078
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.50      1.00      0.67         1
         18       0.00      0.00      0.00         2
         19       0.00      0.00      0.00         1
         20       0.50      1.00      0.67         1
         21       0.00      0.00      0.00         1
         22       0.00      0.00      0.00         0
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         1
         29       0.00      0.00      0.00         0
         30       0.00      0.00      0.00         0

avg / total       0.12      0.23      0.15        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [295]:
scores = cross_val_score(lr_model3, X4_train, y4_train, cv=8)
print("Cross Validation Score for each K",scores)
scores.mean() 

Cross Validation Score for each K [0.38461538 0.34210526 0.22222222 0.41176471 0.52       0.35
 0.4        0.3       ]


0.3663384469847318

### Custom Stop Words

In [296]:
# create a custom stopwords list - I choose to start with the nltk list
from nltk.corpus import stopwords

nltk_stopwords = stopwords.words("english")
#nltk_stopwords

In [297]:

print(type(nltk_stopwords))
print(len(nltk_stopwords))
my_stopwords = nltk_stopwords + ["trees", "mountain", "little", "little bit", "painting", 'let', 'right', 'maybe', 'brush', 
                                 'like', 'want', 'color', 'paint', 'nice', 'll', 'don', 've', 'tree', 'mountains', 'bush',
                                 'white']
print(len(my_stopwords))

<class 'list'>
179
200


### New Vectors using custom stop words

In [298]:
tfidf3 = TfidfVectorizer(lowercase=True, 
                        stop_words= my_stopwords, 
                        max_df=0.9, 
                        min_df=0.15,
                        use_idf = True,
                        ngram_range = (1,2))

tfidf_dm3 = tfidf3.fit_transform(df2['transcript'])

cv2 = CountVectorizer(lowercase=True, 
                     stop_words= my_stopwords,
                     binary=False,
                     max_df=0.90, 
                     min_df=0.15,
                     ngram_range = (1,2)) 
cv2_dm = cv2.fit_transform(df2['transcript'])

print(tfidf_dm3.shape)
print(cv2_dm.shape)

(245, 1430)
(245, 1430)


In [299]:
# PCA with newest weighted vector

for i in range(0,300, 10):
    
    pca2 = PCA(n_components=i)
    X_pca2 = pca2.fit_transform(tfidf_dm3.toarray())
    #print("Number of components: " + str(i))
    #print('PCA Total Variance Explained: ' + str(sum(pca2.explained_variance_ratio_)))
    # Decided to use 170

In [300]:
# 170 components from above was determined to be the most valuable combination
pca4 = PCA(n_components=170)
X_pca4 = pca4.fit_transform(cv2_dm.toarray())

print( 'PCA Total Variance Explained: ' + str(sum(pca4.explained_variance_ratio_)))

PCA Total Variance Explained: 0.9404247899500614


In [301]:
# PCA with newest count vector

for i in range(0,300, 10):
    
    pca3 = PCA(n_components=i)
    X_pca3 = pca3.fit_transform(cv2_dm.toarray())
    #print("Number of components: " + str(i))
    #print('PCA Total Variance Explained: ' + str(sum(pca3.explained_variance_ratio_)))
    
    # Decided to use 130

In [302]:
# 170 components from above was determined to be the most valuable combination
pca3 = PCA(n_components=130)
X_pca3 = pca3.fit_transform(cv2_dm.toarray())

print( 'PCA Total Variance Explained: ' + str(sum(pca3.explained_variance_ratio_)))

PCA Total Variance Explained: 0.8836610257324878


#### Since we are trying to reduce the feature space we are going to use the PCA components derived from the count vector from now on.

In [303]:
X5 = X_pca3  #remember this is the output from the vectorizer and we are turning it into an array
#print(type(X4), X4[0:10])
#print(X5.shape)

y5 = df2['Season'].values #this is an array of labels
#print(type(y4), y4[0:10])

In [304]:
# train and test on count
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.05, random_state=52) #random_state is set seed

# function creates 5 output structures - order matters
print(X5_train.shape)
print(X5_test.shape)
print(y5_train.shape)
print(y5_test.shape)

(232, 130)
(13, 130)
(232,)
(13,)


### Predictions using PCA derived from count vector using custom stop words.
### Decision Tree then Random Forest

In [305]:
#Decision Tree with PCA

# fit a Decision Tree dt_model_1 to the data
dt_model_2 = DecisionTreeClassifier(random_state=42)
#print(dt_model_1)
dt_model_2.fit(X5_train, y5_train)

# make predictions
clf8_expected = y5_test
clf8_predicted = dt_model_2.predict(X5_test)

#print(dt_model_1.score(X_test, y_test))

# summarize the fit of the dt_model_1
print("accuracy: \n" + str(metrics.accuracy_score(clf8_expected, clf8_predicted)))
print('===================')
print(metrics.classification_report(clf8_expected, clf8_predicted))

accuracy: 
0.15384615384615385
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         2
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
          6       0.50      0.33      0.40         3
         18       0.50      0.50      0.50         2
         20       0.00      0.00      0.00         0
         21       0.00      0.00      0.00         1
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         26       0.00      0.00      0.00         0
         27       0.00      0.00      0.00         0
         30       0.00      0.00      0.00         0
         31       0.00      0.00      0.00         1

avg / total       0.19      0.15      0.17        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [306]:
# Random Forest using weights
rf5_model = RandomForestClassifier(random_state=42)

#print(rf5_model)

rf5_model.fit(X5_train, y5_train)

rf5_expected = y5_test
rf5_predicted = rf5_model.predict(X5_test)

#print(rf5_model.score(X4_test,y4_test))

# summarize the fit of the rf5_model
print("accuracy: \n" + str(metrics.accuracy_score(rf5_expected, rf5_predicted)))
print(metrics.classification_report(rf5_expected, rf5_predicted))

accuracy: 
0.07692307692307693
             precision    recall  f1-score   support

          1       1.00      0.50      0.67         2
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         3
         18       0.00      0.00      0.00         2
         20       0.00      0.00      0.00         0
         21       0.00      0.00      0.00         1
         23       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         0
         29       0.00      0.00      0.00         0
         31       0.00      0.00      0.00         1

avg / total       0.15      0.08      0.10        13



C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



# [T3]

### K-Means

In [307]:
from sklearn.cluster import KMeans
My_k = 13
km = KMeans(n_clusters= My_k, init='k-means++', max_iter=100, n_init=1, random_state = 42,
                verbose=True)
km.fit(cv2_dm)
clusters = km.labels_.tolist()

Initialization complete
Iteration  0, inertia 693904.000
Iteration  1, inertia 419453.441
Iteration  2, inertia 415062.441
Iteration  3, inertia 414076.725
Iteration  4, inertia 413527.773
Iteration  5, inertia 413278.544
Iteration  6, inertia 413148.070
Converged at iteration 6: center shift 0.000000e+00 within tolerance 1.497819e-04


In [308]:
# Add cluster to df2

df2['Cluster'] = clusters
df2[['Season', 'Episode', 'Cluster']].head(10)

C:\Users\Paul\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Season,Episode,Cluster
0,29,1,1
1,20,1,1
2,29,6,8
3,6,7,9
4,1,1,12
6,3,10,10
7,21,1,1
8,2,10,10
9,28,1,0
11,2,4,12
